In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

from agage_archive.config import Paths
from agage_archive.io import read_ale_gage
from agage_archive.data_selection import read_release_schedule
from agage_archive.io_other_formats import read_wang
from agage_archive.util import tz_local_to_utc

## Comparison to Ray Wang's ALE/GAGE files

Ray Wang (GA Tech) has previously processed the ALE/GAGE data. There are severeal points in his files that have been flagged following discussions with station PIs over the years. This notebook checks for consistency between the datasets, and outputs rows that can be incorporated into the data_exclude spreadsheet.

Ray also converted the data to the SIO-05 scale (TU-87 for CH4), so this notebook also checks these conversions.

Ray Wang's files are all in station local times (no daylight savings applied). 

In [ ]:
# Use the agage_test output folder to store times to flag (ignored by git)
output_dir = Paths(test=True).output

Define some functions to compare the two datasets

In [ ]:
def read_files(species, site, network,
               data_exclude = False,
               utc = False):
    """ Read the data files for a given species, site and network.
    Return two dataframes, one for Rigby and one for Wang.
    
    Args:
        species (str): Species name
        site (str): Site name
        network (str): Network name
        utc (bool): If True, convert timestamps to UTC.
        data_exclude (bool): If True, apply flagging from data_exclude.
    Returns:
        df (pandas.DataFrame): Data processed using this repo converted to a pandas dataframe
        df_wang (pandas.DataFrame): Ray Wang's version of the data
    """

    df_wang = read_wang(species, site, network, utc=utc)
    df_wang.rename(columns={"mf": "mf_wang"}, inplace=True)

    df = read_ale_gage(species, site, network,
                       utc=utc,
                       data_exclude=data_exclude,
                       scale="default").to_pandas()

    return df, df_wang


def check_timestamps(df, df_wang):
    """Check that the timestamps in df and df_wang are the same
    
    Args: 
        df (pandas.DataFrame): Data processed by this repository
    """
    df_timestamps = set(df.index)
    df_wang_timestamps = set(df_wang.index)
    df_only_timestamps = df_timestamps - df_wang_timestamps
    df_wang_only_timestamps = df_wang_timestamps - df_timestamps

    if df_only_timestamps:
        print(f"Timestamps only in df: {df_only_timestamps}")

    if df_wang_only_timestamps:
        print(f"Timestamps only in df_wang: {df_wang_only_timestamps}")

    # Check timestamps are the same
    if len(df) != len(df_wang):
        raise ValueError("Dataframes are different lengths. Check for duplicates")

    return df_only_timestamps, df_wang_only_timestamps


def compare_scales(df, df_wang, species, site, plot = False):
    """ Compare scales of two dataframes."""

    df_merged = pd.concat([df, df_wang], axis=1)

    if plot:
        (df_merged["mf"]/df_merged["mf_wang"]).plot(marker = ".",
                                                    ylim = [0.99, 1.01],
                                                    ylabel = "mf/mf_wang",
                                                    label=f"{species} - {site}")

    print(f"Scale difference Rigby/Wang: {(df_merged['mf']/df_merged['mf_wang']).mean()}")

    # Find where ratio is more than 0.1% different
    ratio = df_merged["mf"]/df_merged["mf_wang"] - 1
    ratio = ratio[ratio.abs() > 0.001]
    if len(ratio) > 0:
        print("!!!!!! More than 1% difference:")
        print(f".... {ratio}")


def list_missing_periods(file_handle, flagged_in_wang, df, site, species, network):
    """Group missing periods into date ranges. These periods will be converted to UTC

    Args:
        file_handle (file_handle): File handle to redirect output that can be pasted into data_exclude.xlsx
        flagged_in_wang (pandas.DatetimeIndex): Points that are flagged in Wang, but not in df
        df (pandas.Dataframe): Dataframe from read_ale_gage (converted from xarray)
        site (str): Site code
        species (str): Species name
        network (str): Network, either ALE or GAGE
    """

    df_wang_flagged = df[["mf"]].copy()
    df_wang_flagged["flag"] = False
    df_wang_flagged.loc[flagged_in_wang, "flag"] = True
    # Remove rows where df_wang_flagged["mf"] is NaN
    df_wang_flagged = df_wang_flagged.dropna(subset=["mf"])
    # Move the index to a column "time"
    df_wang_flagged = df_wang_flagged.reset_index()

    # Identify ranges where consecutive flagged values exist
    flagged_ranges = []
    start_range = None
    for idx, row in df_wang_flagged.iterrows():
        if row['flag']:
            if start_range is None:
                start_range = row["time"]
        else:
            if start_range is not None:
                flagged_ranges.append((start_range, df_wang_flagged.loc[idx-1, "time"]))
                start_range = None

    # If the last range extends till the end of the dataframe
    if start_range is not None:
        flagged_ranges.append((start_range, df_wang_flagged.loc[len(df_wang_flagged)-1, "time"]))

    if len(flagged_ranges) > 0:
        for flagged_range in flagged_ranges:
            # Convert strings to YYYY-MM-DD HH:MM format, separated by a comma
            # Convert to UTC, since exclusion applied after UTC conversion
            # Add a minute before and after to ensure data are removed
            flagged_start = tz_local_to_utc(flagged_range[0], site) - pd.Timedelta(minutes=1)
            flagged_end = tz_local_to_utc(flagged_range[1], site) + pd.Timedelta(minutes=1)
            data_exclude_string = f"{species},{network},"
            data_exclude_string += f"{flagged_start.strftime('%Y-%m-%d %H:%M')},"
            data_exclude_string += f"{flagged_end.strftime('%Y-%m-%d %H:%M')},"
            data_exclude_string += "Flagged by Ray Wang"
            print(data_exclude_string)
            file_handle.write(data_exclude_string + "\n")


def flagged(df, df_wang, utc = False, site = None, verbose = True):
    """ Find values that are NaN in df_wang, but not in df or vice versa
    
    Args:
        df (pandas.Dataframe): Dataframe converted from read_ale_gage output
        df_wang (pandas.Dataframe): Dataframe from Ray Wang's data files
        utc (bool, optional): Convert to UTC
        site (str, optional): site code (required if utc=True)
        verbose (bool, optional): Verbose output
    
    Return: 
        pandas.DatetimeIndex: indices that are NaN in df_wang, but not df
        pandas.DatetimeIndex: indices that are NaN in df, but not df_wang
    
    """

    def flagged_compare(df1, df2):
        """ Find values that are NaN in df1, but not in df2 """
        flagged_in_1_not_2 = df1.isna() & df2.notna()
        indices = flagged_in_1_not_2[flagged_in_1_not_2 == True].index
        if len(indices) > 0:
            if utc:
                indices = tz_local_to_utc(indices, site)
            for i in indices:
                if verbose:
                    print(i.strftime("%Y-%m-%d %H:%M"))
        
        return indices

    if verbose:
        print("Wang flagged:")
    flagged_in_wang = flagged_compare(df_wang["mf_wang"], df["mf"])

    if verbose:
        print("Rigby flagged:")
    flagged_in_rigby = flagged_compare(df["mf"], df_wang["mf_wang"])

    return flagged_in_wang, flagged_in_rigby

Wrapper function to run all of the above for a particular network

In [ ]:
def run_network(network,
                utc = False, data_exclude = False, plot = False):
    """Wrapper to run each of the above functions for one network

    Args:
        network (str): Network, either ALE or GAGE
        utc (bool, optional): Convert times to UTC. Defaults to False.
        data_exclude (bool, optional): Flag out data between date ranges in data_exclude.xlsx. Defaults to False.
        plot (bool, optional): Plot Wang/Rigby data points. Defaults to False.

    Returns:
        pandas.DataFrame: Dataframe based on read_ale_gage output
        pandas.DataFrame: Dataframe from Ray Wang's files
    """

    # Read release schedule to get sites and species
    rs = read_release_schedule(network)

    with open(output_dir / f"{network}_wang_to_flag.txt", "w") as f:

        for site in rs.columns:
            f.write(f"{site}................................\n")
            for species in rs.index:
                print(species, site)
                df, df_wang = read_files(species, site, network,
                                        utc = utc, data_exclude = data_exclude)
                check_timestamps(df, df_wang)
                compare_scales(df, df_wang, species, site, plot=plot)
                flagged_in_wang, flagged_in_rigby = flagged(df, df_wang, verbose = True)
                print("Lines for data_exclude:")
                list_missing_periods(f, flagged_in_wang, df, site, species, network)
                print("      ")
                print("**********************")
    
    if plot:
        # Show legend
        plt.legend()
        plt.show()
    
    return df, df_wang

### Compare data versions with no flagging from read_ale_gage

Compare the two datasets, without any additional data flagging in read_ale_gage. This will output dates that need to be flagged by read_ale_gage, based on the values that have been flagged by Ray Wang.

Re-run for ALE and GAGE

In [ ]:
df, df_wang = run_network("GAGE", utc=False, data_exclude=False, plot=False)

### Now compare the two datasets after exclusions from data_exclude have been applied in read_ale_gage

If the two datasets are the same, no times should be output below

In [ ]:
df, df_wang = run_network("GAGE", utc=True, data_exclude=True, plot=True)